We will train a new model that is able to recognize fresh and rotten fruit; Using some combination of transfer learning, data augmentation, and fine tuning.

## The Dataset

The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification). The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that our model will require an output layer of 6 neurons to do the categorization successfully. We'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

We'll start with a model pretrained on ImageNet. We'll load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. The images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue.

In [2]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


## Freeze Base Model

Freezing the base model. This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [3]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model.The last dense layer should have the correct number of neurons to classify the different types of fruit.

In [5]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs,outputs)

In [6]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. 

In [7]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Augment the Data

Now, we'll augment the data to improve the dataset. [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created).

In [9]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                       target_size=(224,224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [10]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10)

Epoch 1/10
37/36 [==============================] - 29s 792ms/step - loss: 2.5433 - accuracy: 0.4611 - val_loss: 1.0170 - val_accuracy: 0.7082
Epoch 2/10
37/36 [==============================] - 20s 532ms/step - loss: 0.7598 - accuracy: 0.7860 - val_loss: 0.4977 - val_accuracy: 0.8480
Epoch 3/10
37/36 [==============================] - 20s 529ms/step - loss: 0.3738 - accuracy: 0.8680 - val_loss: 0.3219 - val_accuracy: 0.8906
Epoch 4/10
37/36 [==============================] - 20s 528ms/step - loss: 0.2645 - accuracy: 0.9078 - val_loss: 0.2575 - val_accuracy: 0.9453
Epoch 5/10
37/36 [==============================] - 20s 527ms/step - loss: 0.1887 - accuracy: 0.9382 - val_loss: 0.1863 - val_accuracy: 0.9544
Epoch 6/10
37/36 [==============================] - 20s 529ms/step - loss: 0.1365 - accuracy: 0.9543 - val_loss: 0.1640 - val_accuracy: 0.9453
Epoch 7/10
37/36 [==============================] - 20s 528ms/step - loss: 0.1120 - accuracy: 0.9585 - val_loss: 0.1835 - val_accuracy: 0.9483

## Unfreeze Model for Fine Tuning

In [11]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = 0.00001),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=2)

Epoch 1/2
37/36 [==============================] - 35s 940ms/step - loss: 0.1249 - accuracy: 0.9662 - val_loss: 0.1463 - val_accuracy: 0.9666
Epoch 2/2
37/36 [==============================] - 21s 560ms/step - loss: 0.0626 - accuracy: 0.9797 - val_loss: 0.1800 - val_accuracy: 0.9696


## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [13]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

11/10 [================================] - 4s 337ms/step - loss: 0.1542 - accuracy: 0.9544


[0.15415623784065247, 0.954407274723053]

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your validation data iterator is called `valid_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [14]:
from run_assessment import run_assessment

In [15]:
run_assessment(model, valid_it)

Evaluating model 5 times to obtain average accuracy...

11/10 [================================] - 4s 345ms/step - loss: 0.1895 - accuracy: 0.9514

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9587.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.


## Generate a Certificate

If you passed the assessment, please return to the course page (shown below) and click the "ASSESS TASK" button, which will generate your certificate for the course.

<img src="./images/assess_task.png" style="width: 800px;">